# Twitter and Tableau

## 1.    Live Twitter Feed  - code written by Alex Ross - Please copy and paste this onto a server with python 3 (I have tested this on 3.7

#### I have implemented this on a Redhat free tier AWS instance.  I also installed postgres on the same instance.  When i ran the python script i used the command (nohup python myscript.py &)  to run it in a background.  Please see Alex Ross's blog for further details (NB: it was originally created for Python 2, i have made subtle changes in the code below to make it work for python 3.

In [222]:
from IPython.core.display import display, HTML
display(HTML("""<a href="https://tableaujunkie.com/post/135404208188/creating-a-live-twitter-feed">Creating a Live Twitter Feed</a>"""))

In [ ]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
from textblob import TextBlob
import json
import psycopg2

#connect to database
try:
    conn = psycopg2.connect("dbname='<dbname>' user='<username>' host='<host>' password='<password>'")
except:
    print ("I am unable to connect to the database")

#consumer key, consumer secret, access token, access secret.
ckey='<customer key>'
csecret='<customer secret>'
atoken='<access token>'
asecret='<access secret>'

#lookup arrays to convert user's timezone to a country
TimeZones = ['International Date Line West','Midway Island','American Samoa','Hawaii','Alaska','Pacific Time (US & Canada)','Tijuana','Mountain Time (US & Canada)','Arizona','Chihuahua','Mazatlan','Central Time (US & Canada)','Saskatchewan','Guadalajara','Mexico City','Monterrey','Central America','Eastern Time (US & Canada)','Indiana (East)','Bogota','Lima','Quito','Atlantic Time (Canada)','Caracas','La Paz','Santiago','Newfoundland','Brasilia','Buenos Aires','Montevideo','Georgetown','Greenland','Mid-Atlantic','Azores','Cape Verde Is.','Dublin','Edinburgh','Lisbon','London','Casablanca','Monrovia','UTC','Belgrade','Bratislava','Budapest','Ljubljana','Prague','Sarajevo','Skopje','Warsaw','Zagreb','Brussels','Copenhagen','Madrid','Paris','Amsterdam','Berlin','Bern','Rome','Stockholm','Vienna','West Central Africa','Bucharest','Cairo','Helsinki','Kyiv','Riga','Sofia','Tallinn','Vilnius','Athens','Istanbul','Minsk','Jerusalem','Harare','Pretoria','Kaliningrad','Moscow','St. Petersburg','Volgograd','Samara','Kuwait','Riyadh','Nairobi','Baghdad','Tehran','Abu Dhabi','Muscat','Baku','Tbilisi','Yerevan','Kabul','Ekaterinburg','Islamabad','Karachi','Tashkent','Chennai','Kolkata','Mumbai','New Delhi','Kathmandu','Astana','Dhaka','Sri Jayawardenepura','Almaty','Novosibirsk','Rangoon','Bangkok','Hanoi','Jakarta','Krasnoyarsk','Beijing','Chongqing','Hong Kong','Urumqi','Kuala Lumpur','Singapore','Taipei','Perth','Irkutsk','Ulaanbaatar','Seoul','Osaka','Sapporo','Tokyo','Yakutsk','Darwin','Adelaide','Canberra','Melbourne','Sydney','Brisbane','Hobart','Vladivostok','Guam','Port Moresby','Magadan','Srednekolymsk','Solomon Is.','New Caledonia','Fiji','Kamchatka','Marshall Is.','Auckland','Wellington',"Nuku'alofa",'Tokelau Is.','Chatham Is.','Samoa','Europe/London','America/New_York','EST']
Country = ['United States Minor Outlying Islands','United States Minor Outlying Islands','American Samoa','United States','United States','United States','Mexico','United States','United States','Mexico','Mexico','United States','Canada','Mexico','Mexico','Mexico','Guatemala','United States','United States','Colombia','Peru','Peru','Canada','Venezuela','Bolivia','Chile','Canada','Brazil','Argentina','Uruguay','Guyana','Greenland','South Georgia and the South Sandwich Islands','Portugal','Cape Verde','Ireland','United Kingdom','Portugal','United Kingdom','Morocco','Liberia','#N/A','Serbia','Slovakia','Hungary','Slovenia','Czech Republic','Bosnia and Herzegovina','Macedonia','Poland','Croatia','Belgium','Denmark','Spain','France','Netherlands','Germany','Germany','Italy','Sweden','Austria','Algeria','Romania','Egypt','Finland','Ukraine','Latvia','Bulgaria','Estonia','Lithuania','Greece','Turkey','Belarus','Israel','Zimbabwe','South Africa','Russia','Russia','Russia','Russia','Russia','Kuwait','Saudi Arabia','Kenya','Iraq','Iran','Oman','Oman','Azerbaijan','Georgia','Armenia','Afghanistan','Russia','Pakistan','Pakistan','Uzbekistan','India','India','India','India','Nepal','Bangladesh','Bangladesh','Sri Lanka','Kazakhstan','Russia','Myanmar','Thailand','Thailand','Indonesia','Russia','China','#N/A','Hong Kong','China','Malaysia','Singapore','Taiwan','Australia','Russia','Mongolia','South Korea','Japan','Japan','Japan','Russia','Australia','Australia','Australia','Australia','Australia','Australia','Australia','Russia','Guam','Papua New Guinea','Russia','Russia','Solomon Islands','New Caledonia','Fiji','Russia','Marshall Islands','New Zealand','New Zealand','Tonga','Tokelau','New Zealand','Samoa','United Kingdom','United States','United States']

#listner for twitter stream
class listener(StreamListener):
        def on_data(self, data):
           # print(data)
            all_data = json.loads(data)

            try:
                tweet = TextBlob(all_data["text"])
                print(tweet.raw)

                # determine if sentiment is positive, negative, or neutral
                if tweet.sentiment.polarity < 0:
                    sentiment = "negative"
                elif tweet.sentiment.polarity == 0:
                    sentiment = "neutral"
                else:
                    sentiment = "positive"

                created_at = all_data["created_at"]
                favorite_count = all_data["favorite_count"]
                favorited = all_data["favorited"]
                filter_level = all_data["filter_level"]
                id_str = all_data["id_str"]
                lang = all_data["lang"]
                retweet_count = all_data["retweet_count"]
                retweeted = all_data["retweeted"]
                source = all_data["source"]
                timestamp_ms = all_data["timestamp_ms"]
                truncated = all_data["truncated"]
                user_description = all_data["user"]["description"]
                user_favourites_count = all_data["user"]["favourites_count"]
                user_followers_count = all_data["user"]["followers_count"]
                user_friends_count = all_data["user"]["friends_count"]
                user_id_str = all_data["user"]["id_str"]
                user_location = all_data["user"]["location"]
                user_name = all_data["user"]["name"]
                user_profile_image_url = all_data["user"]["profile_image_url"]
                user_screen_name = all_data["user"]["screen_name"]
                user_statuses_count = all_data["user"]["statuses_count"]
                user_time_zone = all_data["user"]["time_zone"]

                #convert hastags array into comma separated string
                hashtags = all_data["entities"]["hashtags"]
                hashtags_str = ""
                for i in range(0,len(hashtags),1):
                    hashtags_str = str(hashtags_str) + hashtags[i]["text"]
                    if i + 1 < len(hashtags):
                        hashtags_str = hashtags_str + ","

                #convert urls into comma separated strinf
                urls = all_data["entities"]["urls"]
                urls_str = ""
                for i in range(0,len(urls),1):
                    urls_str = str(urls_str) + urls[i]["url"]
                    if i + 1 < len(urls):
                        urls_str = urls_str + ","

                #convert user timezone into country
                user_timezone_country = ''
                if user_time_zone in TimeZones:
                    index = TimeZones.index(user_time_zone)
                    user_timezone_country= Country[index]
                c = conn.cursor()

                #insert data into database
                c.execute("INSERT INTO Twitter (tweet,created_at,timestamp_ms,favorite_count,favorited,filter_level,id_str,lang,retweet_count,retweeted,source,truncated,"
                          "user_description,user_favourites_count,user_followers_count,user_friends_count,user_id_str,user_location,user_name,user_profile_image_url,"
                          "user_screen_name,user_statuses_count,user_time_zone,hashtags,urls,user_timezone_country,polarity,sentiment) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)",
                    (tweet.raw,created_at,timestamp_ms,favorite_count,favorited,filter_level,id_str,lang,retweet_count,retweeted,source,truncated,user_description,user_favourites_count,
                     user_followers_count,user_friends_count,user_id_str,user_location,user_name,user_profile_image_url,user_screen_name,user_statuses_count,user_time_zone,hashtags_str,urls_str,user_timezone_country,tweet.sentiment.polarity,sentiment))

                conn.commit()

                return(True)

            except Exception as e:
                print(e)
                print (all_data)

            def on_error(self, status):
                print (status)


auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)


while True:
    try:
        twitterStream = Stream(auth, listener())
        twitterStream.filter(track=['@ExpoNHS','#ExpoNHS','#Expo19NHS','@Expo19NHS']) #insert keywords for Twitter search
        #twitterStream.sample() #random sample from twitter

    except:
        # Oh well, reconnect and keep going
        continue

# 2.   Load 7 days of Tweets from the Twitter Rest API - and search based on data loaded in prep.
##### Beware - usage limits apply

### Below is a dataframe to test the function with before using with prep

In [31]:
import pandas as pd
search = pd.DataFrame([('@ExpoNHS OR #ExpoNHS OR #Expo19NHS OR @Expo19NHS','event',10),('Tableau','Tableau',10)])
search.columns=(['keywords','subject','rows'])

### Below is the final code that will work with prep - the dataframe will be the passed from prep.  In this example, the code is looking for a dataframe which has the following columns:  keywords, subject and rows.

In [ ]:
import tweepy
import csv
import pandas as pd
import json
from textblob import TextBlob
import datetime as dt


def twitterfeed(search):

    consumer_key="<customer_key>"
    consumer_secret="<customer_secret>"
    access_token="<access_token>"
    access_token_secret="<access_token_secret>"

    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)


    tweetall = pd.DataFrame()
    for subject in search.index:
        tweetsdf = pd.DataFrame()
    
        for tweet in tweepy.Cursor(api.search,q=search.keywords[subject], lang="en",count=10).items(search.rows[subject]):
       
            tweets=pd.DataFrame()
            tweets['created_at'] = [tweet.created_at]
            tweets['created_at'] = tweets.created_at.map(lambda x: dt.datetime.strftime(x, '%Y-%m-%dT%H:%M:%SZ'))
            tweets['tweettext'] = [tweet.text]
            tweets['polarity'] = [TextBlob(tweet.text).sentiment.polarity]
            if TextBlob(tweet.text).sentiment.polarity < 0:
                tweets['sentiment'] = "negative"
            elif TextBlob(tweet.text).sentiment.polarity > 0:
                tweets['sentiment'] = "positive"
            else:
                tweets['sentiment'] = "neutral"
            
            tweets['source'] = [tweet.source]
            tweets['id'] = [tweet.id]
            tweets['name']= pd.io.json.json_normalize(tweet.user._json).head(1).name
            tweets['screen_name']=pd.io.json.json_normalize(tweet.user._json).head(1).screen_name
            tweets['location']=pd.io.json.json_normalize(tweet.user._json).head(1).location
            tweets['followers']=pd.io.json.json_normalize(tweet.user._json).head(1).followers_count
            tweets['friends'] = pd.io.json.json_normalize(tweet.user._json).head(1).friends_count
            tweets['user_description']=pd.io.json.json_normalize(tweet.user._json).head(1).description
            try:
                tweets['hashtag']= pd.io.json.json_normalize(pd.io.json.json_normalize(tweet.entities).hashtags[0]).head(1).text
            except:
                tweets['hashtag']=''
    
            tweetsdf = tweetsdf.append(tweets)
        tweetsdf['subject']=search.subject[subject]
        tweetall = tweetall.append(tweetsdf)
    
    tweetall = tweetall.reset_index(drop = True)
    return tweetall

def get_output_schema():
    return pd.DataFrame({
            'created_at' : prep_datetime(),
            'tweettext' : prep_string(),
            'polarity' : prep_decimal(),
            'sentiment' : prep_string(),
            'source' : prep_string(),
            'id' : prep_int(),
            'name' : prep_string(),
            'screen_name': prep_string(),
            'location' : prep_string(),
            'followers' : prep_int(),
            'friends' : prep_int(),
            'user_description': prep_string(),
            'hashtag': prep_string(),
            'subject' : prep_string()
        });

# Lets look at how this works:

### The following code connects to twitter:

In [3]:
#the following libraries are imported

import tweepy
import csv
import pandas as pd
import json
from textblob import TextBlob
import datetime as dt

### Here, you enter your API key details from twitter.  You need to sign up for a twitter developer account and fill in the form to request approval.

In [ ]:


consumer_key="<CUSTOMER KEY GOES HERE>"
consumer_secret="< CUS SECRET GOES HERE>"
access_token="<ACCESS TOKEN GOES HERE>"
access_token_secret="<ACCESS TOKEN SECRET GOES HERE>"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

### Next, to test that your twitter script will work in prep, you need to simulate the dataframe with the required fields that the script needs.  I have created one called search.  In prep, the column names you will pass need to be the same.

In [25]:
search = pd.DataFrame([('@ExpoNHS OR #ExpoNHS OR #Expo19NHS OR @Expo19NHS','event',2),('Tableau OR ironviz','Tableau Tweets',2)])
search.columns=(['keywords','subject','rows'])

search

,keywords,subject,rows
0,@ExpoNHS OR #ExpoNHS OR #Expo19NHS OR @Expo19NHS,event,2
1,Tableau OR ironviz,Tableau Tweets,2


### this will search twitter accross each subject specified in the previous dataframe.  In this case it will make 2 searches.  It will also only bring back the number of rows as per specified in the dataframe - in this case its 10 

In [27]:
for subject in search.index:
    tweetsdf = pd.DataFrame()


    for tweet in tweepy.Cursor(api.search,q=search.keywords[subject], lang="en",count=10).items(search.rows[subject]):
        print (tweet.text)


RT @NHSEngland: .@HopeGorton discusses body image, mental health and social media in her new blog post: https://t.co/PVcEHnPscA

Join her a…
RT @Pers_Care: Good to work with @JohnMcArts planning for a creative session at #Expo19NHS.  And love this blog from @ace_national   What c…
RT @dfombnr: Our Forest Department Tableau stood 4th best in the #IndependenceDayIndia celebrations held in the parade grounds of Mahabubna…
Quick somebody who is better at data visualization than me make a tableau chart analyzing all the Twitter data on T… https://t.co/F2JQZRiYZU


## Now we have tested the twitter connection, we will now add the data we want to a pandas dataframe

In [ ]:
tweetall = pd.DataFrame()
for subject in search.index:
    tweetsdf = pd.DataFrame()
    
    for tweet in tweepy.Cursor(api.search,q=search.keywords[subject], lang="en",count=10).items(search.rows[subject]):
       
        tweets=pd.DataFrame()
        tweets['created_at'] = [tweet.created_at]
        tweets['created_at'] = tweets.created_at.map(lambda x: dt.datetime.strftime(x, '%Y-%m-%dT%H:%M:%SZ'))
        tweets['tweettext'] = [tweet.text]
        
        #Note: this part uses Text Blob to check for sentement
        
        tweets['polarity'] = [TextBlob(tweet.text).sentiment.polarity]
        if TextBlob(tweet.text).sentiment.polarity < 0:
            tweets['sentiment'] = "negative"
        elif TextBlob(tweet.text).sentiment.polarity > 0:
            tweets['sentiment'] = "positive"
        else:
            tweets['sentiment'] = "neutral"
            
        tweets['source'] = [tweet.source]
        tweets['id'] = [tweet.id]
        
        #the info below is contained in nested json.  Json Normalize is a great tool to help with this.
        
        tweets['name']= pd.io.json.json_normalize(tweet.user._json).head(1).name
        tweets['screen_name']=pd.io.json.json_normalize(tweet.user._json).head(1).screen_name
        tweets['location']=pd.io.json.json_normalize(tweet.user._json).head(1).location
        tweets['followers']=pd.io.json.json_normalize(tweet.user._json).head(1).followers_count
        tweets['friends'] = pd.io.json.json_normalize(tweet.user._json).head(1).friends_count
        tweets['user_description']=pd.io.json.json_normalize(tweet.user._json).head(1).description
        
        #Because the availability of a hashtag field is variable - depending on whether one actually exists or not, 
        #i have added an exception clause of blank.  Also we are only displaying the first hashtag per tweet. If you would like to 
        #expand that further, you could by using the json normalize function to create an additional hashtag table.  
        #However, this will increase the number of rows.
        
        try:
            tweets['hashtag']= pd.io.json.json_normalize(pd.io.json.json_normalize(tweet.entities).hashtags[0]).head(1).text
        except:
            tweets['hashtag']=''
    
    tweetsdf = tweetsdf.append(tweets)
    tweetsdf['subject']=search.subject[subject]
    tweetall = tweetall.append(tweetsdf)
    
tweetall = tweetall.reset_index(drop = True)

tweetall.head(5)

### create the final function that prep asks for
#### To do this i just wrap everything around a function called twitterfeed - i am paramaterising it by search.  That way, it will work with my test dataframe called search, and also with prep as well (as prep will replace search with whatever dataframe it passes through)

In [33]:
def twitterfeed(search):

    consumer_key="<your customer key>"
    consumer_secret="<your customer secret>"
    access_token="<your access token>"
    access_token_secret="<your access token secret>"

    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)


    tweetall = pd.DataFrame()
    for subject in search.index:
        tweetsdf = pd.DataFrame()
    
        for tweet in tweepy.Cursor(api.search,q=search.keywords[subject], lang="en",count=10).items(search.rows[subject]):
       
            tweets=pd.DataFrame()
            tweets['created_at'] = [tweet.created_at]
            tweets['created_at'] = tweets.created_at.map(lambda x: dt.datetime.strftime(x, '%Y-%m-%dT%H:%M:%SZ'))
            tweets['tweettext'] = [tweet.text]
            tweets['polarity'] = [TextBlob(tweet.text).sentiment.polarity]
            if TextBlob(tweet.text).sentiment.polarity < 0:
                tweets['sentiment'] = "negative"
            elif TextBlob(tweet.text).sentiment.polarity > 0:
                tweets['sentiment'] = "positive"
            else:
                tweets['sentiment'] = "neutral"
            
            tweets['source'] = [tweet.source]
            tweets['id'] = [tweet.id]
            tweets['name']= pd.io.json.json_normalize(tweet.user._json).head(1).name
            tweets['screen_name']=pd.io.json.json_normalize(tweet.user._json).head(1).screen_name
            tweets['location']=pd.io.json.json_normalize(tweet.user._json).head(1).location
            tweets['followers']=pd.io.json.json_normalize(tweet.user._json).head(1).followers_count
            tweets['friends'] = pd.io.json.json_normalize(tweet.user._json).head(1).friends_count
            tweets['user_description']=pd.io.json.json_normalize(tweet.user._json).head(1).description
            try:
                tweets['hashtag']= pd.io.json.json_normalize(pd.io.json.json_normalize(tweet.entities).hashtags[0]).head(1).text
            except:
                tweets['hashtag']=''
    
            tweetsdf = tweetsdf.append(tweets)
        tweetsdf['subject']=search.subject[subject]
        tweetall = tweetall.append(tweetsdf)
    
    tweetall = tweetall.reset_index(drop = True)
    return tweetall # this is passing the final dataframe back to prep

# You can test it here

In [37]:
twitterfeed(search).sample(1)

,created_at,tweettext,polarity,sentiment,source,id,name,screen_name,location,followers,friends,user_description,hashtag,subject
6,2019-08-15T08:05:07Z,The #Expo19NHS theatres give you the chance to...,0.1,positive,Hootsuite Inc.,1161911683020992513,NHS England SW,NHSEnglandSW,"South West, England",3766,839,@NHSEngland and @NHSImprovement working togeth...,Expo19NHS,event


### Finally create a prep schema to map the fields returned with fields that prep will expect.

In [30]:
def get_output_schema():
    return pd.DataFrame({
            'created_at' : prep_datetime(),
            'tweettext' : prep_string(),
            'polarity' : prep_decimal(),
            'sentiment' : prep_string(),
            'source' : prep_string(),
            'id' : prep_int(),
            'name' : prep_string(),
            'screen_name': prep_string(),
            'location' : prep_string(),
            'followers' : prep_int(),
            'friends' : prep_int(),
            'user_description': prep_string(),
            'hashtag': prep_string(),
            'subject' : prep_string()
        });

# Split the Tweet into Keywords
### I would like to analyse the words used in the tweets - this is how i did it:

##### Below uses sklearn to look at the data taking into account standard english words.  You can change the stop_words to use other dictionaries if you like.  There is plenty of lists to choose from if you search for them.  The dataframe used will be the previously loaded twitter data in prep.  So this script will be placed AFTER ther previous one.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import re
import pandas as pd

def detlanguage(df):

    dataframe = pd.DataFrame()

    for A in df.index:
        docs = df.head(A+1).tweet.tolist()
        cv=CountVectorizer(max_df=1, stop_words='english', max_features=20000)
        word_count_vector=cv.fit_transform(docs)
        keywords = pd.DataFrame(list(cv.vocabulary_.keys())[:20])
        keywords.columns = ['keyword']
        keywords['id'] = df.id[A]
        keywords['created_at']=df.created_at[A]
        dataframe = dataframe.append(keywords)
    return dataframe



def get_output_schema():
    return pd.DataFrame({
            'keyword' : prep_string(),
            'id' : prep_string(),
            'created_at' : prep_string()
        });



##### Below looks at all the tweets as a whole, and excludes the most common words (i.e the words that are used more than 85% of the time).  If you combine this with the previous script in prep and join on keyword as an INNER join, you will only see the words that are deemed important.  I have also implemented sentiment on each word - which won't be as accurate when taken out of context, but still, this will be useful to see overall sentiment of a specific word.

I have also tagged each word using Penn Treebank tags 
https://www.clips.uantwerpen.be/pages/mbsp-tags


In [ ]:
#summarytest
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk import word_tokenize, pos_tag, ne_chunk
analyser = SentimentIntensityAnalyzer()
import re
import pandas as pd
from textblob import TextBlob

def overallwithsentiment(df):
    print(df.head(1))
    #nltk.download('averaged_perceptron_tagger')
    #nltk.download('punkt')
    def overalllang(df):
        docs = df.tweet.tolist()
        cv=CountVectorizer(max_df=0.85, stop_words='english', max_features=20000)
        word_count_vector=cv.fit_transform(docs)
        keywords = pd.DataFrame(list(cv.vocabulary_.keys())[:100])
        keywords.columns = ['keyword']
        return keywords

    #det_language = detlanguage(stop4)
    #sum_language = overalllang(stop4)
    def print_sentiment_scores(sentence):
        snt = analyser.polarity_scores(sentence)
        return snt


    dataframe = pd.DataFrame()
    for A in overalllang(df).index:
        score1 = pd.io.json.json_normalize(print_sentiment_scores(overalllang(df).keyword[A]))
        score1['keyword'] = overalllang(df).keyword[A]
        score1['Wordtag'] = pd.DataFrame(TextBlob(overalllang(df).keyword[A]).tags)[1]
        dataframe = dataframe.append(score1)
    dataframe = dataframe.reset_index()
    return dataframe


def get_output_schema():
    return pd.DataFrame({
            'neg' : prep_decimal(),
            'neu' : prep_decimal(),
            'pos' : prep_decimal(),
            'compound' : prep_decimal(),
            'keyword' : prep_string(),
            'Wordtag' : prep_string()
        });

#### Finally, I join the original tweet results back to the keywords so i get all the tweet information as well as the keywords.